This is part of the sum of squares project, where I aim to generalize the work done in https://arxiv.org/abs/2005.12312. The arXiv paper deals with Pythagoras numbers and universal quadratic forms in the simplest family of cubic numbers fields (Shanks cubic fields). I start with the Lehmer quintic field, which is the field generated by Emma Lehmer's quintic polynomial $p_n(x)$. The $K_{n}=\mathbb{Q}[x]/\langle p_n(x)\rangle$ are totally real and cyclic.

Crucial in this approach is identifying indecomposables (totally real elements that cannot be expressed as the sum of two totally real elements) in algebraic number fields. However, we must switch over to Colmez units (https://www.sciencedirect.com/science/article/pii/S0022314X12000844), that can be used to describe a fundamental domain for the action of totally positive elements in the Minkowiski space. Below is an attempt to find such a basis. They key was to leverage symmetry. We have that $\{u_0, u_1, u_2, u_3\}$ is a Colmez set where

\begin{aligned}
u_0 &= \frac{\rho_0^2}{\rho_2^2} \\
u_1 &= \frac{\rho_1^2}{\rho_2^2} \\
u_2 &= \left( \frac{\rho_3^2}{\rho_2^2} \right)^{-1} = \frac{\rho_2^2}{\rho_3^2} \\
u_3 &= \left( \frac{\rho_4^2}{\rho_2^2} \right)^{-1} = \frac{\rho_2^2}{\rho_4^2},
\end{aligned}

where the $\rho_0$ is a give root of the Lehmer quintic, and the other $\rho_i$ are its Galois conjugates. If the cell below outputs all $1$'s, then the Colmez condition is satisfied. Now, this can be used to indentify indecompsables in $K_n$. However, the expressions for the $u_i$ in the power basis are quite complicated, so I am exploring comparing this number field with another 'simpler one'.

In [9]:
import sympy as sp

#rho is a root of the Lehmer quintic p_n(x). In

rho, n, x = sp.symbols('rho n x')

#Definition of the Lehmer quintic p_n(x).

poly = rho**5 + n**2 * rho**4 - 2*(n**3+3*n**2+5*n+5)*rho**3+(n**4+5*n**3+11*n**2+15*n+5)*rho**2+(n**3+4*n**2+10*n+10)*rho+1

#Value of the n parameter.

n_val = 10

#The generator of the Galois group.

def sigma(val, nv):
    return (nv+2+nv*val-val**2)/(1+(nv+2)*val)

#A concrete Lehmer polynomial and its roots to 10000 decimal points.
poly_eval = poly.subs(n, n_val)

raw_roots = [r.as_real_imag()[0] for r in sp.nroots(poly_eval, n=10000)]

#Ordering the roots.

base_rho = max(raw_roots)


ordered_roots = [base_rho]
for i in range(4):
    next_root = sigma(ordered_roots[-1], n_val)
    actual_root = min(raw_roots, key=lambda x: abs(x - next_root))
    ordered_roots.append(actual_root)

roots = ordered_roots

#Computing the fundamental units for n=2. These are the Galois conjugates of rho in terms of {1, rho, rho^2, rho^3, rho^4, rho^5}.

# Row 3 expressions
p31 = n**4 + 2*n**3 + 3*n**2 + 4*n + 4
p32 = 2*n**2 + 4*n + 4
p33 = (n + 2) * (n**2 + n + 2)
p34 = (n + 2)**2
p35 = (n + 2) * (n**2 + n + 2)

# Row 4 expressions
p41 = -(n**2 + 3*n + 3) * (n**4 + n**3 + 4*n**2 + 2*n + 1)
p42 = -(n + 1) * (2*n**3 + 4*n**2 + 6*n + 3)
p43 = -(n + 1)**2 * (n**3 + 2*n**2 + 3*n + 3)
p44 = -(n + 1)**2 * (n**2 + 3*n + 3)
p45 = -(n**5 + 4*n**4 + 9*n**3 + 13*n**2 + 9*n + 3)

# Row 5 expressions
p51 = n**8 + 6*n**7 + 21*n**6 + 50*n**5 + 86*n**4 + 105*n**3 + 99*n**2 + 72*n + 32
p52 = 2*n**6 + 10*n**5 + 30*n**4 + 60*n**3 + 83*n**2 + 72*n + 32
p53 = n**7 + 6*n**6 + 20*n**5 + 47*n**4 + 80*n**3 + 94*n**2 + 72*n + 32
p54 = n**6 + 7*n**5 + 24*n**4 + 51*n**3 + 75*n**2 + 72*n + 32
p55 = n**7 + 6*n**6 + 21*n**5 + 49*n**4 + 80*n**3 + 93*n**2 + 72*n + 32

# Construct the matrix
M = sp.Matrix([
    [1, 1, 1, 1, 1],
    [-n**2, 0, 0, 0, 0],
    [p31, p32, p33, p34, p35],
    [p41, p42, p43, p44, p45],
    [p51, p52, p53, p54, p55]
])

M = -(n_val**2)*M.inv().subs(n, n_val)

matrix_list = [[0 for _ in range(5)] for _ in range(5)]

for i in range(5):
  for j in range(5):
    matrix_list[i][j] = sp.Rational(sp.together(M[i, j]))

fundamental_units = []
for i in range(1, 5):
  combined = 0
  for j in range(5):
    combined = rho**j*matrix_list[i][j]+combined
  fundamental_units.append(combined)

rhoB1 = fundamental_units[0]
rhoB2 = fundamental_units[1]
rhoB3 = fundamental_units[2]
rhoB4 = fundamental_units[3]

def simplification(expr):
    f = poly_eval
    expanded_expr = sp.expand(expr)
    reduced = sp.rem(expanded_expr, f, rho)
    return reduced

def inverted(inpu):
    given = simplification(inpu)
    X = sp.symbols('x0:5')
    inv_poly = sum(X[i] * rho**i for i in range(5))
    product = simplification((given * inv_poly).expand())
    prod_poly = sp.Poly(product, rho)
    coeffs = [prod_poly.coeff_monomial(rho**i) for i in range(5)]
    system = [coeffs[0] - 1] + coeffs[1:]
    sol = sp.solve(system, X)
    if not sol:
        raise ValueError("Matrix is singular: Element is not invertible in this field.")

    return sum(sol[X[i]] * rho**i for i in range(5))

#Defining the Colmez units.
inv_rho0 = inverted(rho**2)
inv_rho1 = inverted(rhoB1**2)
inv_rho2 = inverted(rhoB2**2)
inv_rho3 = inverted(rhoB3**2)
inv_rho4 = inverted(rhoB4**2)


u0 = simplification(rho**2 * inv_rho2)   #u0 = (rho_0^2/rho_2^2)
u1 = simplification(rhoB1**2 * inv_rho2) #u1 = (rho_1^2/rho_2^2)
u2 = simplification(inv_rho3 * rhoB2**2) #u2 = (rho_3^2/rho_2^2)^{-1}
u3 = simplification(inv_rho4 * rhoB2**2) #u3 = (rho_4^2/rho_2^2)^{-1}

units = [u0, u1, u2, u3]

def specific_product_row(index, sigma):
    prd = sp.S(1)
    row = []
    for i in range(index):
        prd = simplification(prd * units[sigma[i]])
    for j in range(5):
        row.append(prd.subs(rho, roots[j]).evalf(1000))
    return row

def get_full_matrix(sigma):
    matrix_data = []
    for i in range(5):
        matrix_data.append(specific_product_row(i, sigma))
    return sp.Matrix(matrix_data)

def get_full_matrix_det(sigma):
  mat = get_full_matrix(sigma)
  det = mat.det()
  if det > 0:
        return 1
  else:
        return -1

all_perms = [
    [0, 1, 2, 3], [0, 1, 3, 2], [0, 2, 1, 3], [0, 2, 3, 1], [0, 3, 1, 2], [0, 3, 2, 1],
    [1, 0, 2, 3], [1, 0, 3, 2], [1, 2, 0, 3], [1, 2, 3, 0], [1, 3, 0, 2], [1, 3, 2, 0],
    [2, 0, 1, 3], [2, 0, 3, 1], [2, 1, 0, 3], [2, 1, 3, 0], [2, 3, 0, 1], [2, 3, 1, 0],
    [3, 0, 1, 2], [3, 0, 2, 1], [3, 1, 0, 2], [3, 1, 2, 0], [3, 2, 0, 1], [3, 2, 1, 0]
]
perm_signs = [
     1, -1, -1,  1,  1, -1,  # Starts with 0
    -1,  1,  1, -1, -1,  1,  # Starts with 1
     1, -1, -1,  1,  1, -1,  # Starts with 2
    -1,  1,  1, -1, -1,  1   # Starts with 3
]

#If the following list produces 24 identical elements, then the units as defined are Colmez.

for i in range(24):
  print(i, perm_signs[i]*get_full_matrix_det(all_perms[i]))

0 1
1 1
2 1
3 1
4 1
5 1
6 1
7 1
8 1
9 1
10 1
11 1
12 1
13 1
14 1
15 1
16 1
17 1
18 1
19 1
20 1
21 1
22 1
23 1


In [8]:
# Definining the fundamental units for n=2. These are the Galois conjugates of rho in terms of {1, rho, rho^2, rho^3, rho^4, rho^5}. Computed in cell below.


rhoB1 = sp.Rational(284/55) + sp.Rational(486/11)*rho + sp.Rational(-227/11)*rho**2 + sp.Rational(12/11)*rho**3 + sp.Rational(16/55)*rho**4
rhoB2 = (
    sp.Rational(-28, 55) +
    sp.Rational(-104, 11) * rho +
    sp.Rational(78, 11) * rho**2 +
    sp.Rational(-8, 11) * rho**3 +
    sp.Rational(-7, 55) * rho**4
)
rhoB3 = (
    sp.Rational(-653, 55) +
    sp.Rational(-348, 11) * rho +
    sp.Rational(173, 11) * rho**2 +
    sp.Rational(-9, 11) * rho**3 +
    sp.Rational(-12, 55) * rho**4
)
rhoB4 = (
    sp.Rational(177, 55) +
    sp.Rational(-45, 11) * rho +
    sp.Rational(-24, 11) * rho**2 +
    sp.Rational(5, 11) * rho**3 +
    sp.Rational(3, 55) * rho**4
)



for i in range(5):
    for j in range(5):
        combined = sp.together(M[i, j])
        num, den = sp.fraction(combined)
        expanded_fraction = sp.expand(num) / sp.expand(den)
        matrix_list[i][j] = f"${sp.latex(expanded_fraction)}$"



In [3]:
u0 = (sp.Rational(661, 55)*rho**4 -
      sp.Rational(959, 11)*rho**3 +
      sp.Rational(1532, 11)*rho**2 +
      sp.Rational(634, 11)*rho +
      sp.Rational(59, 55))

u1 = (sp.Rational(1247368190624555222081818356799229, 278883132050210468329274705182720)*rho**4 +
      sp.Rational(699743131326048861814100092266907, 111553252820084187331709882073088)*rho**3 -
      sp.Rational(25535553690997564190070394734007809, 111553252820084187331709882073088)*rho**2 +
      sp.Rational(104758645602860813923680833054063913, 223106505640168374663419764146176)*rho +
      sp.Rational(833931553263742214943952913799174751, 4462130112803367493268395282923520))

u2 = (sp.Rational(-2596, 5)*rho**4 -
      1913*rho**3 +
      36935*rho**2 -
      81831*rho -
      sp.Rational(8039, 5))

u3 = (sp.Rational(11871, 11)*rho**4 +
      sp.Rational(98474, 11)*rho**3 -
      sp.Rational(407993, 11)*rho**2 -
      sp.Rational(149904, 11)*rho -
      sp.Rational(2764, 11))

units = [u0, u1, u2, u3]


In [ ]:
print(simplification(u0),
      simplification(u0*u1),
      simplification(u0*u1*u2),
      simplification(u0*u1*u2*u3)


)

661*rho**4/55 - 959*rho**3/11 + 1532*rho**2/11 + 634*rho/11 + 59/55 307429650011664958271832453119720332093*rho**4/892426022560673498653679056584704 - 2229046794386667670337797470480289379047*rho**3/892426022560673498653679056584704 + 444752576376790713012745091926133630305*rho**2/111553252820084187331709882073088 + 736578583314585203270927815068986786027*rho/446213011280336749326839528292352 + 27326247070082495331350492801848567059/892426022560673498653679056584704 -225621524431023631087653767955458009*rho**4/4462130112803367493268395282923520 - 14689535155444494205103969780635341*rho**3/55776626410042093665854941036544 + 1867644841123449910486832816497044421*rho**2/446213011280336749326839528292352 - 8094953061732626459182084453829702217*rho/892426022560673498653679056584704 - 795394975016803989797417384327068031/4462130112803367493268395282923520 -13157752266514296194768386876794831*rho**4/405648192073033408478945025720320 - 9138147454154173888880280349477849*rho**3/8112963841460668

In [ ]:

# Simplification rules for higher powers of rho.
simp8 = n ** 6 + 4 * n ** 5 + 13 * n ** 4 + 25 * n ** 3 + 31 * n ** 2 + 15 * n + rho ** 4 * (
        n ** 8 + 6 * n ** 7 + 24 * n ** 6 + 64 * n ** 5 + 128 * n ** 4 + 189 * n ** 3 + 226 * n ** 2 + 190 * n + 90) + rho ** 3 * (
                -2 * n ** 9 - 15 * n ** 8 - 67 * n ** 7 - 205 * n ** 6 - 458 * n ** 5 - 753 * n ** 4 - 910 * n ** 3 - 770 * n ** 2 - 400 * n - 101) + rho ** 2 * (
                n ** 10 + 9 * n ** 9 + 44 * n ** 8 + 148 * n ** 7 + 358 * n ** 6 + 636 * n ** 5 + 797 * n ** 4 + 650 * n ** 3 + 236 * n ** 2 - 50 * n - 75) + rho * (
                n ** 9 + 8 * n ** 8 + 39 * n ** 7 + 127 * n ** 6 + 301 * n ** 5 + 518 * n ** 4 + 623 * n ** 3 + 474 * n ** 2 + 190 * n + 40) + 5
simp7 = -n ** 4 - 2 * n ** 3 - 6 * n ** 2 - 10 * n + rho ** 4 * (
        -n ** 6 - 4 * n ** 5 - 13 * n ** 4 - 25 * n ** 3 - 31 * n ** 2 - 15 * n - 5) + rho ** 3 * (
                2 * n ** 7 + 11 * n ** 6 + 39 * n ** 5 + 97 * n ** 4 + 174 * n ** 3 + 221 * n ** 2 + 190 * n + 90) + rho ** 2 * (
                -n ** 8 - 7 * n ** 7 - 27 * n ** 6 - 76 * n ** 5 - 157 * n ** 4 - 250 * n ** 3 - 280 * n ** 2 - 200 * n - 51) + rho * (
                -n ** 7 - 6 * n ** 6 - 24 * n ** 5 - 64 * n ** 4 - 130 * n ** 3 - 199 * n ** 2 - 200 * n - 100) - 10
simp6 = n ** 2 + rho ** 4 * (n ** 4 + 2 * n ** 3 + 6 * n ** 2 + 10 * n + 10) + rho ** 3 * (
        -2 * n ** 5 - 7 * n ** 4 - 15 * n ** 3 - 21 * n ** 2 - 15 * n - 5) + rho ** 2 * (
                n ** 6 + 5 * n ** 5 + 11 * n ** 4 + 14 * n ** 3 + n ** 2 - 10 * n - 10) + rho * (
                n ** 5 + 4 * n ** 4 + 10 * n ** 3 + 10 * n ** 2 - 1)
simp5 = -n ** 2 * rho ** 4 + rho ** 3 * (2 * n ** 3 + 6 * n ** 2 + 10 * n + 10) - rho ** 2 * (
        n ** 4 + 5 * n ** 3 + 11 * n ** 2 + 15 * n + 5) - rho * (n ** 3 + 4 * n ** 2 + 10 * n + 10) - 1

simplification_list = [simp5, simp6, simp7, simp8]


def get_reduction_dict(par):
    return {
        rho ** 5: simplification_list[0].subs(n, par),
        rho ** 6: simplification_list[1].subs(n, par),
        rho ** 7: simplification_list[2].subs(n, par),
        rho ** 8: simplification_list[3].subs(n, par)
    }


In [ ]:
from sympy import factorint

# Take the denominator of your u1 coefficient (v2[0] for example)
den = 892426022560673498653679056584704

# Get the prime factorization
print(factorint(den))

{2: 105, 11: 1}
